# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [35]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

fatal: destination path 'SQuAD-explorer' already exists and is not an empty directory.


In [36]:
import json

In [37]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [38]:
type(train_data)

dict

In [39]:
list(train_data.keys())

['data', 'version']

In [40]:
type(train_data["data"])

list

In [41]:
len(train_data["data"])

442

In [42]:
type(train_data["data"][0])

dict

In [43]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [44]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [45]:
len(train_data["data"][0]["paragraphs"])

55

In [46]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [47]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

In [48]:
import pandas as pd

In [49]:
x1 = pd.DataFrame(train_data)

In [50]:
x2 = pd.DataFrame(train_data['data'])

In [51]:
x3 = pd.DataFrame(train_data["data"][0]["paragraphs"])

In [52]:
x1.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [53]:
x2.head()

,title,paragraphs
0,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ..."
1,Beyoncé,[{'context': 'Beyoncé Giselle Knowles-Carter (...
2,Montana,[{'context': 'Montana i/mɒnˈtænə/ is a state i...
3,Genocide,"[{'context': 'The phrase ""in whole or in part""..."
4,Antibiotics,[{'context': 'The emergence of resistance of b...


In [54]:
x3.head()

,context,qas
0,"Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S..."
1,"As at most other universities, Notre Dame's st...","[{'answers': [{'answer_start': 248, 'text': 'S..."
2,The university is the major seat of the Congre...,"[{'answers': [{'answer_start': 119, 'text': 'R..."
3,The College of Engineering was established in ...,"[{'answers': [{'answer_start': 487, 'text': 'e..."
4,All of Notre Dame's undergraduate students are...,"[{'answers': [{'answer_start': 496, 'text': 'L..."


In [55]:
# Flatten SQuAD-like JSON into a list of dicts
rows = []
for article in train_data["data"]:
    title = article["title"]
    for para in article["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            question = qa["question"]
            qid = qa["id"]
            
            # some questions have multiple answers
            for ans in qa["answers"]:
                rows.append({
                    "title": title,
                    "context": context,
                    "question": question,
                    "answer_text": ans["text"],
                    "answer_start": ans["answer_start"],
                    "id": qid
                })

# Convert to DataFrame
df = pd.DataFrame(rows)

In [56]:
df.head()

,title,context,question,answer_text,answer_start,id
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,5733be284776f41900661182
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,5733be284776f4190066117f
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,5733be284776f41900661180
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,5733be284776f41900661181
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,5733be284776f4190066117e


In [57]:
df.shape

(87599, 6)

In [58]:
count_multi = sum(len(qa["answers"]) > 1 
                  for article in train_data["data"]
                  for para in article["paragraphs"]
                  for qa in para["qas"])
print(f"Questions with >1 answer: {count_multi}")

Questions with >1 answer: 0


In [59]:
# Flatten SQuAD-like JSON into a list of dicts
rows = []
for article in train_data["data"]:
    title = article["title"]
    for para in article["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            question = qa["question"]
            qid = qa["id"]
            for ans in qa["answers"]:
                answer_start = ans["answer_start"]
                answer_text = ans["text"]      
                rows.append({
                "title": title,
                "context": context,
                "question": question,
                "id": qid,
                "answer_text": answer_text,
                "answer_start": answer_start
            })    


# Convert to DataFrame
df = pd.DataFrame(rows)

In [60]:
df.head()

,title,context,question,id,answer_text,answer_start
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,5733be284776f41900661182,Saint Bernadette Soubirous,515
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,5733be284776f4190066117f,a copper statue of Christ,188
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,5733be284776f41900661180,the Main Building,279
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,5733be284776f41900661181,a Marian place of prayer and reflection,381
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,5733be284776f4190066117e,a golden statue of the Virgin Mary,92


In [61]:
df.shape

(87599, 6)

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [62]:
# YOUR CHANGES HERE

rows = []
for article in train_data["data"]:
    title = article.get("title")
    para_n = article.get("paragraphs", [])
    for p_idx, para in enumerate(para_n):
    # for para in article["paragraphs"]:
        context = para["context"]
    #     for qa in para["qas"]:
    #         question = qa["question"]
    #         qid = qa["id"]
    #         for ans in qa["answers"]:
    #             answer_start = ans["answer_start"]
    #             answer_text = ans["text"]      
        rows.append({
                "document_title": title,
                "paragraph_index": p_idx,
                "paragraph_context": context
        })    


# Convert to DataFrame
df = pd.DataFrame(rows)

In [63]:
df.shape

(18896, 3)

In [64]:
df.head()

,document_title,paragraph_index,paragraph_context
0,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,2,The university is the major seat of the Congre...
3,University_of_Notre_Dame,3,The College of Engineering was established in ...
4,University_of_Notre_Dame,4,All of Notre Dame's undergraduate students are...


In [65]:
df.to_csv("parsed.tsv", sep="\t", index=False)

Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [66]:
# YOUR CHANGES HERE

from sklearn.feature_extraction.text import CountVectorizer


In [67]:
vectorizer = CountVectorizer()


In [69]:
vectorizer.fit(df["paragraph_context"].fillna(""))


,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [70]:
X_bow = vectorizer.transform(df["paragraph_context"].fillna(""))


In [71]:
X_bow.shape


(18896, 78537)

In [72]:
import numpy as np

In [73]:
np.random.seed(42)  
word_hash = np.random.randint(0, 2, size=(len(vectorizer.vocabulary_), 1024)) * 2 - 1

In [74]:
word_hash

array([[-1,  1, -1, ..., -1,  1,  1],
       [ 1,  1,  1, ...,  1,  1, -1],
       [-1, -1,  1, ...,  1, -1,  1],
       ...,
       [-1, -1, -1, ...,  1, -1, -1],
       [-1, -1,  1, ..., -1, -1, -1],
       [-1,  1,  1, ...,  1,  1,  1]], shape=(78537, 1024))

In [75]:
X_hash = X_bow @ word_hash   
X_hash.shape


(18896, 1024)

In [76]:
X_hash

array([[-38,  32,   4, ...,  -8,   0,  30],
       [-50,  28,  12, ...,  14, -24,   6],
       [-38,  14,   2, ...,  -2, -10,  22],
       ...,
       [-22,  10,  16, ...,   2,  20, -12],
       [-39,  -7, -17, ...,   9,  17,  15],
       [-12,   2,  10, ...,   0,  12,  14]], shape=(18896, 1024))

In [77]:
def vec_to_json(v):
    return json.dumps([int(x) for x in v])  # integers are fine here

df_out = df[["document_title", "paragraph_index"]].copy()
df_out["paragraph_vector_json"] = [vec_to_json(row) for row in X_hash]


In [78]:
df_out.head()

,document_title,paragraph_index,paragraph_vector_json
0,University_of_Notre_Dame,0,"[-38, 32, 4, 14, -8, -16, -8, -2, -4, -28, 20,..."
1,University_of_Notre_Dame,1,"[-50, 28, 12, 26, -10, -12, -36, 20, -4, -18, ..."
2,University_of_Notre_Dame,2,"[-38, 14, 2, 8, -12, 0, -12, -14, 2, -14, -12,..."
3,University_of_Notre_Dame,3,"[-28, 30, 42, 24, -12, 22, -8, 16, -12, -34, 2..."
4,University_of_Notre_Dame,4,"[-1, 5, 7, 21, -1, -11, -11, -7, -21, -5, 9, 1..."


In [79]:
df_out.to_csv("paragraph-vectors.tsv.gz", sep="\t", index=False)


Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [ ]:
# YOUR CHANGES HERE

...

Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [80]:
# YOUR CHANGES HERE

questions = pd.read_csv("questions.tsv", sep="\t")

In [81]:
questions.head()

,question_id,question
0,1,What was the goal of the abuse of region project?
1,4,How many satellites in the Beidou-1 constellat...
2,7,When did Beyoncé receive ten nominations for ...
3,10,"With which goddess did Sulla, Pompey, and Juli..."
4,13,What area is considered to have a desert clima...


Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [ ]:
# YOUR CHANGES HERE

...

Submit "question-vectors.tsv" in Gradescope.

In [83]:
X_q_bow = vectorizer.transform(questions["question"].fillna(""))
X_q_bow

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 990 stored elements and shape (100, 78537)>

In [ ]:


X_q_hash = X_q_bow @ word_hash
X_q_hash.shape

def vec_to_json(v):
    return json.dumps([int(x) for x in v])

df_out = pd.DataFrame({
    "question_id": questions["question_id"],
    "question_vector_json": [vec_to_json(row) for row in X_q_hash]
})

df_out.to_csv("question-vectors.tsv", sep="\t", index=False)



In [85]:
df_out.head()

,question_id,question_vector_json
0,1,"[0, -4, 2, 0, -4, 0, -6, -4, -8, -2, 8, 0, -6,..."
1,4,"[-3, 3, 3, 1, -1, 1, -3, -1, -3, -1, -5, -3, 1..."
2,7,"[-2, -6, 4, 2, 2, -6, 4, 0, 2, 4, -2, 0, -8, -..."
3,10,"[-7, 5, -1, -1, -7, 7, 1, 1, 5, -1, -3, 3, 5, ..."
4,13,"[2, -2, -4, -4, 2, -4, 2, -2, -4, 0, 0, 0, 2, ..."


## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [87]:
# YOUR CHANGES HERE

import json
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# ----------------------------
# 1) Load paragraph vectors
# ----------------------------
para = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t")
P = np.vstack(para["paragraph_vector_json"].apply(lambda s: np.array(json.loads(s), dtype=float)).to_numpy())
# P.shape -> (n_paragraphs, 1024)

# ----------------------------
# 2) Load question vectors
# ----------------------------
qs = pd.read_csv("question-vectors.tsv", sep="\t")
Q = np.vstack(qs["question_vector_json"].apply(lambda s: np.array(json.loads(s), dtype=float)).to_numpy())
# Q.shape -> (n_questions, 1024)

# ----------------------------
# 3) Nearest neighbors (Euclidean)
# ----------------------------
k = 5
nn = NearestNeighbors(n_neighbors=k, metric="euclidean", algorithm="brute")
nn.fit(P)
dists, idxs = nn.kneighbors(Q, n_neighbors=k)

# ----------------------------
# 4) Build results table
# ----------------------------
rows = []
for qi, qid in enumerate(qs["question_id"].to_list()):
    for rank in range(k):
        pi = idxs[qi, rank]
        rows.append({
            "question_id": qid,
            "rank": rank + 1,
            "document_title": para.loc[pi, "document_title"],
            "paragraph_index": int(para.loc[pi, "paragraph_index"])
        })

matches = pd.DataFrame(rows, columns=["question_id", "question_rank", "document_title", "paragraph_index", "distance"])

# Save (gzipped is nice for big outputs)
matches.to_csv("question-paragraph-matches.tsv", sep="\t", index=False)



Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [ ]:
# YOUR CHANGES HERE

...

Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.